In [27]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('Migrant Requirements.csv')
df.head()
# df.shape

,Could you please specify your age range?,What is your educational background?,Could you please indicate the length of your professional experience in your field?,Have you ever used the services of a migration agent before?,"Could you please simply specify the type of visa you are currently holding? (e.g., Subclass 500)",Would you prefer to book a migration agent online or schedule an in-person meeting?,How important do you think it is to have a digital platform that matches migrants with the right migration agents?
0,20-24,Bachelor (Undergraduate),< 1 year,No,188A,In-person Meeting,5
1,20-24,Bachelor (Undergraduate),NaN,No,student visa,In-person Meeting,3
2,20-24,Bachelor (Undergraduate),1 - 3 years,No,subclass 500,In-person Meeting,5
3,20-24,Bachelor (Undergraduate),NaN,Yes,500,Online,5
4,16-20,Diploma,NaN,Yes,500,Online,4


In [31]:
columnList = np.array(df.columns)
print(columnList)

['Could you please specify your age range?'
 'What is your educational background?'
 'Could you please indicate the length of your professional experience in your field?'
 'Have you ever used the services of a migration agent before?'
 'Could you please simply specify the type of visa you are currently holding? (e.g., Subclass 500)'
 'Would you prefer to book a migration agent online or schedule an in-person meeting?'
 'How important do you think it is to have a digital platform that matches migrants with the right migration agents?  ']


In [33]:
df.isnull().sum()

Could you please specify your age range?                                                                                 0
What is your educational background?                                                                                     0
Could you please indicate the length of your professional experience in your field?                                     32
Have you ever used the services of a migration agent before?                                                             0
Could you please simply specify the type of visa you are currently holding? (e.g., Subclass 500)                         1
Would you prefer to book a migration agent online or schedule an in-person meeting?                                      0
How important do you think it is to have a digital platform that matches migrants with the right migration agents?       0
dtype: int64

In [35]:
df = df.rename(columns={
    'Could you please specify your age range?': 'Age Range',
    'What is your educational background?': 'Education',
    'Could you please indicate the length of your professional experience in your field?': 'Experience',
    'Have you ever used the services of a migration agent before?': 'Used Agent Before',
    'Could you please simply specify the type of visa you are currently holding? (e.g., Subclass 500)': 'Visa Type',
    'Would you prefer to book a migration agent online or schedule an in-person meeting?': 'Booking Preference',
    'How important do you think it is to have a digital platform that matches migrants with the right migration agents?  ': 'Platform Importance'
})
df.isnull().sum()

Age Range               0
Education               0
Experience             32
Used Agent Before       0
Visa Type               1
Booking Preference      0
Platform Importance     0
dtype: int64

In [45]:
df_copy = df.copy()
# df_copy.shape
df_copy['Education'].unique()

array(['Bachelor (Undergraduate)', 'Diploma', 'Year 12 and below',
       'Master', 'Doctorate'], dtype=object)

In [41]:
keywords = ['citizen', 'moved', 'naturalized', 'decades', 'years', 'permanent resident']


mask = ~df['Visa Type'].str.lower().str.contains('|'.join(keywords), na=False)


df_cleaned = df[mask].copy()

df_cleaned.shape


(93, 7)

In [43]:
agents = pd.read_csv('../PrdData/agents_filled.csv')
agents.shape
agent_cols = agents[['MARN', 'Full_name', 'Business_name']]

target_rows = agents.shape[0]  #  4534

# df_copy
df_expanded = df_cleaned.copy()

rows_to_add = target_rows - df_expanded.shape[0]

if rows_to_add > 0:
    empty_rows = pd.DataFrame([{}] * rows_to_add, columns=df_expanded.columns)
    df_expanded = pd.concat([df_expanded, empty_rows], ignore_index=True)

agent_cols = agents[['MARN', 'Full_name', 'Business_name']].reset_index(drop=True)
df_expanded = df_expanded.reset_index(drop=True)
df_final = pd.concat([agent_cols, df_expanded], axis=1)

df_final.shape
df_final.columns

Index(['MARN', 'Full_name', 'Business_name', 'Age Range', 'Education',
       'Experience', 'Used Agent Before', 'Visa Type', 'Booking Preference',
       'Platform Importance'],
      dtype='object')

In [28]:
new_columns = [
    'Apply Visa Type', 
    'Has Health Issues', # yes or no
    'Character Requirements', # pass or no
    'Occupation', 
    'English Proficiency', 
    'Children', 
    'Skills Assessment',
    'Financial Capacity', 
    # 'Regional Migration Restrictions',
    'Dependents', # yes or no
    'Residence History', # years of living in austrlian
    'Health Insurance', # yes or no 
    'Marital status', # single / engaged / married
    'Preferred State', 
    'Nationality',
    'Prior Visa Compliance', # spectific country  
]

for col in new_columns:
    df_final[col] = None

df_final.head()

,MARN,Full_name,Business_name,Age Range,Education,Experience,Used Agent Before,Visa Type,Booking Preference,Platform Importance,...,Children,Skills Assessment,Financial Capacity,Dependents,Residence History,Health Insurance,Marital status,Preferred State,Nationality,Prior Visa Compliance
0,1800328,Kumar Rahul,Blue Stone Migration,20-24,Bachelor (Undergraduate),< 1 year,No,188A,In-person Meeting,5.0,...,None,None,None,None,None,None,None,None,None,None
1,1799100,Ramandeep Kaur,Expert Migration Services,20-24,Bachelor (Undergraduate),NaN,No,student visa,In-person Meeting,3.0,...,None,None,None,None,None,None,None,None,None,None
2,1799035,Jaspreet Kaur,Nextwave Migration,20-24,Bachelor (Undergraduate),1 - 3 years,No,subclass 500,In-person Meeting,5.0,...,None,None,None,None,None,None,None,None,None,None
3,2318090,Balwant Kaur,Harmony immigration services,20-24,Bachelor (Undergraduate),NaN,Yes,500,Online,5.0,...,None,None,None,None,None,None,None,None,None,None
4,1570947,Nahida,OZimmi Solutions,16-20,Diploma,NaN,Yes,500,Online,4.0,...,None,None,None,None,None,None,None,None,None,None


In [40]:
import pandas as pd

current_year = 2025
agents = pd.read_csv('../PrdData/agents_filled.csv')

invalid_marn = agents[agents['MARN'].astype(str).str.len() == 6]
print(len(invalid_marn))
print(invalid_marn[['MARN', 'Full_name']])

1255
        MARN                     Full_name
13    104705         Aaron Wai Kwong Yeung
21    427601              Abdul Hasib Khan
22    637156            Abdul Nasir Khaled
23    960056             Abdul Nadeem Qazi
36    636459                        Ada Ho
...      ...                           ...
4525  426480             John Graham Adams
4526  702049        Katherine Van Greuning
4528  640631  Margot Sabine Bettina Obrist
4529  532994              Mark Peter Welch
4532  848168          Paula Louise McGeown

[1255 rows x 2 columns]


In [46]:

def format_marn(marn):
    marn_str = str(marn)
    if len(marn_str) == 7:
        return marn_str        
    elif len(marn_str) < 7:
        return marn_str.zfill(7)  
    else:
        return marn_str       

agents['MARN'] = agents['MARN'].apply(format_marn)


In [50]:

invalid_marn = agents[agents['MARN'].astype(str).str.len() != 7]

print(len(invalid_marn))
print(invalid_marn[['MARN', 'Full_name']])

0
Empty DataFrame
Columns: [MARN, Full_name]
Index: []


In [54]:
agents['start_year'] = agents['MARN'].astype(str).str[:2].astype(int)
agents['start_year'] = agents['start_year'].apply(lambda x: 2000 + x if x < current_year % 100 + 1 else 1900 + x)


agents['Experience_years'] = current_year - agents['start_year']

# print(agents[['MARN', 'Full_name', 'Experience_years']].head())

agents.to_csv('agents_data.csv', index=False)


In [30]:
df_final.isnull().sum()

MARN                         0
Full_name                    0
Business_name                0
Age Range                 4441
Education                 4441
Experience                4471
Used Agent Before         4441
Visa Type                 4442
Booking Preference        4441
Platform Importance       4441
Apply Visa Type           4534
Has Health Issues         4534
Character Requirements    4534
Occupation                4534
English Proficiency       4534
Children                  4534
Skills Assessment         4534
Financial Capacity        4534
Dependents                4534
Residence History         4534
Health Insurance          4534
Marital status            4534
Preferred State           4534
Nationality               4534
Prior Visa Compliance     4534
dtype: int64

In [44]:
from faker import Faker
import random

fake = Faker("en_AU")
num_rows = len(df_final)

age_ranges = ['< 16', '16-20', '20-24', '25-33', '33-40', '40+']
weights =    [0.02,   0.15,    0.45,    0.2,     0.1,    0.08] 

df_final['Age Range'] = df_final['Age Range'].apply(
    lambda x: x if pd.notnull(x) else random.choices(age_ranges, weights=weights)[0]
)



df_final['Education'] = df_final['Education'].apply(
    lambda x: x if pd.notnull(x) else random.choice(['High School', 'Bachelor', 'Master', 'PhD'])
)

df_final['Experience'] = df_final['Experience'].apply(
    lambda x: x if pd.notnull(x) else random.choice(['0', '< 1 year', '1 - 3 years', '> 5 years'])
)

df_final['Used Agent Before'] = df_final['Used Agent Before'].apply(
    lambda x: x if pd.notnull(x) else random.choice(['Yes', 'No'])
)

df_final['Visa Type'] = df_final['Visa Type'].apply(
    lambda x: x if pd.notnull(x) else random.choice(['500', '485', '190', '888'])
)

df_final['Booking Preference'] = df_final['Booking Preference'].apply(
    lambda x: x if pd.notnull(x) else random.choice(['Online', 'In-person Meeting'])
)

df_final['Platform Importance'] = df_final['Platform Importance'].apply(
    lambda x: x if pd.notnull(x) else random.randint(1, 5)
)

# df_final['Health Requirements'] = df_final['Health Requirements'].apply(
#     lambda x: x if pd.notnull(x) else random.choice(['Yes', 'No'])
# )

# df_final['Regional Migration Restrictions'] = df_final['Regional Migration Restrictions'].apply(
#     lambda x: x if pd.notnull(x) else random.choice(['Yes', 'No'])
# )



df_final['Apply Visa Type'] = random.choices(['Subclass 485', 'Subclass 500', 'Subclass 190', 'Subclass 820'], k=num_rows)
df_final['Has Health Issues'] = random.choices(['yes', 'no'], k=num_rows)
df_final['Character Requirements'] = random.choices(['pass', 'fail'], k=num_rows)
df_final['Occupation'] = [fake.job() for _ in range(num_rows)]
df_final['English Proficiency'] = random.choices(['Basic', 'Intermediate', 'Advanced', 'Fluent'], k=num_rows)
df_final['Children'] = [random.randint(0, 3) for _ in range(num_rows)]
df_final['Skills Assessment'] = random.choices(['pass', 'fail'], k=num_rows)
df_final['Financial Capacity'] = random.choices(['Low', 'Medium', 'High'], k=num_rows)
df_final['Dependents'] = random.choices(['yes', 'no'], k=num_rows)
df_final['Residence History'] = [random.randint(0, 10) for _ in range(num_rows)]
df_final['Health Insurance'] = random.choices(['yes', 'no'], k=num_rows)
df_final['Marital status'] = random.choices(['Single', 'Engaged', 'Married'], k=num_rows)
df_final['Preferred State'] = random.choices(['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'ACT', 'NT'], k=num_rows)
df_final['Nationality'] = [fake.country() for _ in range(num_rows)]
df_final['Prior Visa Compliance'] = random.choices(['Compliant', 'Non-compliant'], k=num_rows)




df_final['Experience'].unique()

array(['< 1 year', '1 - 3 years', '> 5 years', '0', '3 - 5 years'],
      dtype=object)

In [86]:
df_final.isnull().sum()

MARN                               0
Full_name                          0
Business_name                      0
Age Range                          0
Education                          0
Experience                         0
Used Agent Before                  0
Visa Type                          0
Booking Preference                 0
Platform Importance                0
Health Requirements                0
Character Requirements             0
Occupation                         0
English Proficiency                0
Skills Assessment                  0
Financial Capacity                 0
Regional Migration Restrictions    0
Dependents                         0
Residence History                  0
Health Insurance                   0
Marital status                     0
Preferred State                    0
Nationality                        0
Prior Visa Compliance              0
Apply Visa Type                    0
Has Health Issues                  0
Children                           0
d

In [88]:
df_final.to_csv('requirements_filled.csv', index=False)